In [ ]:
#Operaciones algebraicas
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'svg'

#filtrado para suavizar los datos
from scipy.signal import savgol_filter

In [ ]:
from prophet import Prophet
# from fbprophet.deagnostics import cross_validation

# Forecasting Demanda Energía (Prophet)

In [ ]:
# Importo el archivos de datos de consumo de energia en la zona este de EE.UU.
df = pd.read_csv(r'Raw_Data/medidor_1.csv')
df.head()

In [ ]:
print(df.shape)
print('\n')
print(df.info())

In [ ]:
# Cambio de nombre en columnas y eliminamos otra
df.rename(columns={'fechahora':'Datetime', 'demanda_activa':'y[kW]'}, inplace = True)
df.drop(columns='terminal', inplace=True)

df.head()

<b>Target_values: "y[kW]"</b>

In [ ]:
#Convierto a tipo DateTimeIndex la columna "Datetime"
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis = 0, ascending = True, inplace = True)
df.reset_index(inplace = True, drop = True)

df.head()

In [ ]:
# Calcula algunos parámetros estadísticos solo sobre las variables de tipo float/int
df.describe()

## Limpieza de datos

### Eliminación de datos duplicados

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
df.drop_duplicates(subset = 'Datetime', keep = 'last', inplace = True)
df.shape

### Tratamiento de espacios vacios para un grupo de datos continuos

In [ ]:
df_2 = df.set_index('Datetime')
df_2.drop(['2017-08-18 09:15:00'], inplace = True)
df_2

In [ ]:
print(df_2.index.min())
print(df_2.index.max())

In [ ]:
print(f'df_2.index.freq is set to: {df_2.index.freq}')

<i>
Tener un dataset con frecuencia en "None" indica 
que existen datos que perdidos (missing). <br>
Para verificar lo dicho, podemos comparar con un rango de datos
custom e ininterrumpido
</i>

In [ ]:
# Custom range
data_range = pd.date_range(start = min(df_2.index),
                          end = max(df_2.index),
                          freq = '15min') 
#freq = '15min' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
data_range

In [ ]:
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(df_2))}')

In [ ]:
print(data_range.difference(df_2.index))
print('\n')
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(df_2))}')

<i>Lo que ese numero indica es la cantidad de puntos en el tiempo perdidos dentro del conjunto de datos</i>

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (y[kW])
df_3 = df_2.reindex(data_range)

# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
df_3['y[kW]'].interpolate(method='linear', inplace=True)

# Con la interpolación se tiene un datetime (set de hora y dias) continuo
print(f'La df.index.freq ahora es: {df_3.index.freq}, indicando que ya no tenemos valores perdidos')

## Filtro savgol_filter

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_3.index, y=df_3['y[kW]'],
                         mode='lines',
                         name='Datos'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
# Aplica filtro elegido en base a buscar cual mejor se adecua
y_filtered = df_3[["y[kW]"]].apply(savgol_filter,  window_length=5, polyorder=3)

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_3.index,y=df_3['y[kW]'],
                         mode='lines',
                         name='No Filtrada'))
fig.add_trace(go.Scatter(x=y_filtered.index, y=y_filtered['y[kW]'],
                         mode='lines', 
                         name='Filtrada'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

Estudiando la gráfica se observa un comportamiento con patron en temporadas (estación del año). 

In [ ]:
y_filtered.head()

In [ ]:
y_filtered.info()

## EDA: Analisis Exploratorio de Datos

### Extraemos características de la variable Tiempo 

<i>
Podemos dividir la columna de Datetime en sus diferentes componentes. <br>
Esto nos permite encontrar patrones para diferentes grupos.
</i>

In [ ]:
y_filtered['dow'] = y_filtered.index.day_of_week
y_filtered['doy'] = y_filtered.index.day_of_year
y_filtered['year'] = y_filtered.index.year
y_filtered['month'] = y_filtered.index.month
y_filtered['quarter'] = y_filtered.index.quarter
y_filtered['hour'] = y_filtered.index.hour
y_filtered['weekday'] = y_filtered.index.day_name()
y_filtered['woy'] = y_filtered.index.isocalendar().week #week of year
y_filtered['dom'] = y_filtered.index.day # Day of Month
y_filtered['date'] = y_filtered.index.date 

# número de estación del año
y_filtered['season'] = y_filtered['month'].apply(lambda month_number: (month_number%12 + 3)//3) 
# el operador aritmético // solo devuelve a parte entera de la división.

In [ ]:
y_filtered.info()

### Histograma 

In [ ]:
features_num = ["y[kW]"]
y_filtered[features_num].hist(figsize=(10,4)); #el ; es para evitar una grafica duplicada

### Matriz de correlación

In [ ]:
# Calculamos y graficamos
corr_matrix = y_filtered.corr()
sns.heatmap(corr_matrix);

### Box Plot

In [ ]:
sns.boxplot(x = "season", y="y[kW]", data=y_filtered);

### Scatter Plot

In [ ]:
%config InlineBackend.figure_format = "png"
sns.pairplot(
    y_filtered[['y[kW]',
                'season',
                'date',
                'month',
                'year']]
);

In [ ]:
%config InlineBackend.figure_format = "svg"

In [ ]:
# Sacamos las variables no-numéricas
numerical = list(set(y_filtered.columns))

In [ ]:
# Algunas veces se puede analizar una variable ordinal como una numerica. 

# fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15,10))
# for idx, feat in enumerate(numerical):
#     ax = axes[int(idx/4), idx % 4]
#     sns.boxplot(x="season", y=feat, data=y_filtered, ax=ax)
#     ax.set_xlabel("")
#     ax.set_ylabel(feat)
# fig.tight_layout();

### Graficando el consumo de energía a lo largo del tiempo

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_filtered.index, y=y_filtered['y[kW]'],
                         mode='lines', 
                         name='Filtrada'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
# sns.countplot(x="y[kW]", hue="season", data=y_filtered)

### Patrones de fecha y hora

Podemos usar nuestras funciones de fecha y hora extraídas previamente <br>
para ver si surgen patrones recurrentes de los datos agregados. <br>
Tomemos, por ejemplo, la demanda de energía a lo largo del día para cada día de la semana:

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_1 = y_filtered.groupby(['hour', 'weekday'], as_index=False).agg({'y[kW]':'median'})
patron_1

In [ ]:
fig = px.line(patron_1, 
              x = 'hour',
              y = 'y[kW]', 
              color='weekday', 
              title='Mediana de consumo de energia por hs por día de semana ')

fig.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig.show()

In [ ]:
# Dataframe definido para graficar el consumo horario por temporada del año. Mediana de la energía. 
patron_2 = y_filtered.groupby(['hour', 'season'], as_index=False).agg({'y[kW]':'median'})

In [ ]:
fig_2 = px.line(patron_2, 
                x = 'hour',
                y = 'y[kW]', 
                color='season', 
                title='Mediana de consumo de energia por hs por estación')

fig_2.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig_2.show()

## Descompoción de la serie de tiempo

Los puntos que representan datos a lo largo de una serie de tiempo pueden ser interesantes <br>
en cuanto sus patrones se complementes con tendencias de subida/bajada y/o estacionalidad. <br>
Según la info adquirida en el EDA esto parece ser así.

In [ ]:
print(f'El primer punto de medicion fecha/hs es: {min(y_filtered.index)}')
print(f'El último punto de medicion fecha/hs es: {max(y_filtered.index)}')

In [ ]:
# Dataframe de recort
CUTOFF_DATE = pd.to_datetime('2021-09-01')

# Separo df p/ test y df p/ train
train = y_filtered.loc[(y_filtered.index < CUTOFF_DATE)].copy() 
test = y_filtered.loc[y_filtered.index >= CUTOFF_DATE].copy()

In [ ]:
#Se permite recortar varias fechas porque:
#1- El comportamiento es constante en el tiempo.
#2- Alivia la carga de procesamiento en la PC.
print(f'Training shape: {train.shape}\n Testing shape: {test.shape}\n')
print(f'Porcentaje: {(len(test)*100)/len(train):.2f}%\n')
print(f'Las fechas de entrenamiento son: {min(train.index)} & {max(train.index)}')
print(f'Las fechas de test son: {min(test.index)} & {max(test.index)}')